# Applied Seismology, GEOS 626, University of Alaska Fairbanks

- script hw_sumatraB_modes.ipynb
- template script for analyzing modes spectra for sumatra

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import warnings

from obspy import read
from lib_seis import locations_and_tags
from lib_seis import get_dist_az
from lib_seis import station_map_and_table
from lib_seis import sumatra_event
from lib_seis import wf_fft
from lib_seis import w2fstack

In [ ]:
# script settings

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = 10, 5
plt.rcParams['lines.linewidth'] = 1

### Event details
Mw 9.1 2004 Sumatra - Andaman Islands Earthquake \
https://earthquake.usgs.gov/earthquakes/eventpage/official20041226005853450_30

In [ ]:
# fetch and display event details

event_sumatra = sumatra_event()

print()
for i, item in enumerate(event_sumatra):
    print(f'{item} = {event_sumatra[item]}')
print()    

In [ ]:
# setting other data parameters

resample_freq = 1
data_dir      = 'datawf/sumatra_LHZ_acc'

In [ ]:
# read downloaded LHZ acceleration data for selected station locations

st = read(f'{data_dir}/*.sac', header=None)

print('Details of downloaded waveforms - \n')
print(st.__str__(extended=True))
print()

In [ ]:
# station map and distances table for the selected station locations
event_path = f'{data_dir}/event.xml'
inv_path = f'{data_dir}/inv.xml'
station_map_and_table(event_path, inv_path)

In [ ]:
# computing and creating a list of the amplitude spectra of the seismograms for the selected station locations

amps  = []
freqs = []

Nyquist_frequency = resample_freq/2

for tr in st:
    
    tr.detrend('constant')
    tr.detrend('linear')
    tr.taper(max_percentage=0.2, type="cosine")
    
    fft_amp, fft_phase, fft_freq = wf_fft(tr.data, Nyquist_frequency)
    
    freqs.append(fft_freq)
    amps.append(fft_amp)

In [ ]:
# example of stacking spectra

f1 = 0.2 * 1E-3
f2 = 10  * 1E-3
n  = 100000

amp_stack, freq, amplitudes = w2fstack(freqs,amps,f1,f2,n,stack='median')                                                     

plt.figure()
plt.plot(freq*1E3, amp_stack, 'b') 
plt.title('Amplitude spectra stack')
plt.xlabel('Frequency [mHz]')
plt.ylabel('Amplitude [ms^-2 s]')
plt.show()